In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
#tf.enable_eager_execution()

import tensorflow_hub as hub
import os
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [2]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


In [3]:
data_dir = os.path.join('data', 'data')
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')

In [4]:
classes=os.listdir(train_dir)

In [5]:
module_selection = ("inception_v3", 299, 2048) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/2".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(
  MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

BATCH_SIZE = 64 #@param {type:"integer"}

Using https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/2 with input size (299, 299) and output dimension 2048


In [6]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

do_data_augmentation = True #@param {type:"boolean"}
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest' )
else:
  train_datagen = validation_datagen
  
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

Found 74 images belonging to 2 classes.
Found 107 images belonging to 2 classes.


In [ ]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE+(3,),
                                   output_shape=[FV_SIZE])

In [ ]:


do_fine_tuning = False #@param {type:"boolean"}
if do_fine_tuning:
      feature_extractor.trainable = True
  # unfreeze some layers of base network for fine-tuning
      for layer in base_model.layers[-30:]:
        layer.trainable =True
else:
    feature_extractor.trainable = False



In [ ]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
#model.build((None,)+IMAGE_SIZE+(3,))

model.summary()

In [ ]:
#Compile model specifying the optimizer learning rate

LEARNING_RATE = 0.001 #@param {type:"number"}

model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

In [ ]:
EPOCHS=10 #@param {type:"integer"}

history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples//validation_generator.batch_size)

In [ ]:
import matplotlib.pylab as plt
import numpy as np

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()

In [ ]:
import cv2

# Utility
import itertools
import random
from collections import Counter
from glob import iglob


def load_image(filename):
    print(filename)
    img = cv2.imread(filename)
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img


def predict(image):
    probabilities = model.predict(np.asarray([image]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}

In [ ]:



    img = load_image('data/data/train/not_glaucoma/ROI - 983_left.jpeg.png')
    prediction = predict(img)
    print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    



In [ ]:
import time
t = time.time()

export_path = "saved_models/".format(int(t))

In [ ]:

tf.keras.models.save_model(model, export_path)
export_path

In [ ]:
reloaded = tf.keras.models.load_model(export_path, custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
TFLITE_MODEL = "tflite_models/plant_disease_model.tflite"


# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : reloaded(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

# Convert the model to standard TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

In [ ]:


def predict_reload(image):
    probabilities = reloaded.predict(np.asarray([image]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}



In [ ]:


   
    
    img = load_image('data/data/train/not_glaucoma/ROI - 983_left.jpeg.png')
    prediction = predict_reload(img)
    print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    



In [ ]:
from tkinter import *
from tkinter import messagebox 
import matplotlib.pyplot as plt
import pandas

In [ ]:




def get_filenames():
    global path
    path = r"test"
    return os.listdir(path)


def curselect(event):
    global spath
    index = t1.curselection()[0]
    spath = t1.get(index)
    return(spath)


def autoroi(img):

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    thresh = cv2.threshold(gray_img, 130, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=5)

    contours, hierarchy = cv2.findContours(
        thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    biggest = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(biggest)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    roi = img[y:y+h, x:x+w]

    return roi



def predict_reload(image):
    probabilities = reloaded.predict(np.asarray([image]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}



def prediction():

    img=load_image('test/%s' % (spath))


    prob = predict_reload(img)
    print(prob)
    

    return(list(prob.keys())[0])


def run():

    Class = prediction()
    if (Class == 'glaucoma'):
        messagebox.showinfo('Prediction', 'You have been diagnosed with Glaucoma')
    else:
        messagebox.showinfo('Prediction', 'Congratulations! You are Healthy')


def run_all():

    x = os.listdir(path)
    y = []
    affected = 0

    for i in x:
        img=load_image('test/%s' % (spath))
        prob = predict_reload(img)
        Class = 0
        if(prob == 'glaucoma'):
            Class=1
        
        y.append(Class)
        if Class == 1:
            affected += 1

    df = pandas.DataFrame(data=y, index=x, columns=["output"])
    df.to_csv('output.csv', sep=',')


def ROI():
    img = cv2.imread('test/%s' % (spath))
    roi = autoroi(img)
    cv2.imshow("Region of Interest", roi)
def load_image(filename):
    print(filename)
    img = cv2.imread(filename)
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img

def preview():
    img = cv2.imread('test/%s' % (spath))
    cv2.imshow('Image', img)


def graph():

    total = len(os.listdir(path))
    affected = pandas.read_csv('output.csv')
    affected = affected['output'].sum()

    healthy = total - affected

    piey = ["Glaucomatous", "Healthy"]
    piex = [affected, healthy]

    plt.axis("equal")
    plt.pie(piex, labels=piey, radius=1.5, autopct='%0.1f%%', explode=[0.2, 0])
    plt.show()

# Frontend GUI


window = Tk()
window.title("Glaucoma Detection")
window.geometry('1000x550')
window.configure(background='grey')

l1 = Label(window, text="Test Image", font=("Arial", 20), padx=10, bg='grey')
l1.grid(row=0, column=0)

b1 = Button(window, text='Run', font=("Arial", 20), command=run)
b1.grid(row=1, column=3)

b2 = Button(window, text='Preview', font=("Arial", 20), command=preview)
b2.grid(row=1, column=2, rowspan=2, padx=10)

b2 = Button(window, text='ROI', font=("Arial", 20), command=ROI)
b2.grid(row=2, column=2, rowspan=3, padx=10)

b3 = Button(window, text='Run all', font=("Arial", 20), command=run_all)
b3.grid(row=2, column=3)

b4 = Button(window, text='Graph', font=("Arial", 20), command=graph)
b4.grid(row=3, column=3)

t1 = Listbox(window, height=20, width=60, selectmode=SINGLE, font=("Arial", 15), justify=CENTER)
t1.grid(row=1, column=0, rowspan=3, padx=10)
for filename in get_filenames():
    t1.insert(END, filename)
t1.bind('<<ListboxSelect>>', curselect)

sb1 = Scrollbar(window)
sb1.grid(row=1, column=1, rowspan=4)

t1.configure(yscrollcommand=sb1.set)
sb1.configure(command=t1.yview)


window.mainloop()
